In [25]:
import pandas as pd
import pandas.tseries.offsets as offsets


In [26]:
tourism_train = pd.read_csv("../data/train.tsv",delimiter="\t")
print(tourism_train.shape)
tourism_train.head(20)


(5848, 3)


,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
1,2015-01-01,十和田八幡平国立公園,34757
2,2015-01-01,日光国立公園,29714
3,2015-01-01,伊勢志摩国立公園,42652
4,2015-01-01,大山隠岐国立公園,3637
5,2015-01-01,阿蘇くじゅう国立公園,1369
6,2015-01-01,霧島錦江湾国立公園,35352
7,2015-01-01,慶良間諸島国立公園,151
8,2015-01-02,阿寒摩周国立公園,11153
9,2015-01-02,十和田八幡平国立公園,33795


In [ ]:
#観光及び国立公園に関連する語句が含まれる投稿の2015/01/01～2017/12/30までの日別件数（ブログ・twitter・掲示板）データ。
sns = pd.read_csv("../data/nig.tsv",delimiter="\t")

In [4]:
#日光国立公園区画内のSNSロケーション解析データ投稿数データ。
locsns = pd.read_csv("../data/nightley.tsv",delimiter="\t")
print(locsns.shape)
locsns.head()


(730, 3)


,datetime,Japan_count,Foreign_count
0,2016-01-01,28,0
1,2016-01-02,66,1
2,2016-01-03,75,6
3,2016-01-04,50,1
4,2016-01-05,25,0


In [5]:
visitors = pd.read_csv("../data/colopl.tsv",delimiter="\t")
print(visitors.shape)
visitors.head()

(1701, 5)


,year,month,park,country_jp,count
0,2016,1,阿寒摩周国立公園,アメリカ合衆国,1-9
1,2016,1,阿寒摩周国立公園,シンガポール,1-9
2,2016,1,阿寒摩周国立公園,マレーシア,1-9
3,2016,1,阿寒摩周国立公園,中国,1-9
4,2016,1,阿寒摩周国立公園,台湾,19


In [6]:
#国立公園周辺12地点の気象観測データ。
weather = pd.read_csv("../data/weather.tsv",delimiter="\t")
print(weather.shape)
weather.head()

(13140, 24)


,年月日,地点,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),10分間降水量の最大(mm),日照時間(時間),合計全天日射量(MJ/㎡),最深積雪(cm),...,最多風向(16方位),平均蒸気圧(hPa),平均現地気圧(hPa),平均湿度(%),平均海面気圧(hPa),最小相対湿度(%),最低海面気圧(hPa),平均雲量(10分比),天気概況(昼:06時~18時),天気概況(夜:18時~翌日06時)
0,2015/1/1,十和田,-1.9,0.3,-2.9,5.0,1.0,2.1,NaN,18.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/1/2,十和田,-3.9,-1.1,-8.0,0.0,0.0,3.3,NaN,17.0,...,西北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/1/3,十和田,-4.0,-1.6,-8.9,2.0,0.5,1.4,NaN,21.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/1/4,十和田,0.6,3.4,-1.6,0.0,0.0,2.6,NaN,19.0,...,南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/1/5,十和田,0.8,5.4,-5.1,0.0,0.0,4.0,NaN,14.0,...,西南西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#防災科学技術研究所 ⼤⼭鏡ヶ成積雪気象観測施設における積雪気象観測データ。
snow = pd.read_csv("../data/nied_oyama.tsv",delimiter="\t")
print(snow.shape)
snow.head()

(12780, 5)


,日時,積雪深(cm),積雪深重量(kg m^-2),気温(℃),降水量(mm h-1)
0,2015-01-01 00:00:00,99.3,305.1,-4.8,1.0
1,2015-01-01 01:00:00,101.9,308.3,-4.8,1.0
2,2015-01-01 02:00:00,102.5,310.3,-5.3,0.0
3,2015-01-01 03:00:00,101.8,310.6,-5.5,0.0
4,2015-01-01 04:00:00,101.9,311.1,-6.1,0.5


In [8]:
#各国立公園付近エリア単位の「平均人口」の集計値データ
year = 2017
month = 3
population_dic = {}
population_dic[(year, month)] = pd.read_csv("../data/month_time_mesh100m_"+str(year)+"0"+str(month)+".tsv",delimiter="\t")
print(population_dic[(year, month)].shape)
population_dic[(year, month)].head(30)

(265735, 7)


,park,mesh100mid,year,month,dayflag,hour,population
0,伊勢志摩国立公園,51362395334,2017,3,0,16,4
1,伊勢志摩国立公園,51362395334,2017,3,1,9,11
2,伊勢志摩国立公園,51362395334,2017,3,2,9,8
3,伊勢志摩国立公園,51362395334,2017,3,2,16,1
4,伊勢志摩国立公園,51362396400,2017,3,1,0,47
5,伊勢志摩国立公園,51362396400,2017,3,1,1,97
6,伊勢志摩国立公園,51362396400,2017,3,1,2,97
7,伊勢志摩国立公園,51362396400,2017,3,1,3,97
8,伊勢志摩国立公園,51362396400,2017,3,1,4,97
9,伊勢志摩国立公園,51362396400,2017,3,1,5,97


In [9]:
isesima = tourism_train.loc[tourism_train["park"] == "伊勢志摩国立公園",]
print(isesima.shape)
isesima.to_csv("../data/isesima.csv",index=False)
isesima.head()


(731, 3)


,datetime,park,visitors
3,2015-01-01,伊勢志摩国立公園,42652
11,2015-01-02,伊勢志摩国立公園,47035
19,2015-01-03,伊勢志摩国立公園,45368
27,2015-01-04,伊勢志摩国立公園,34960
35,2015-01-05,伊勢志摩国立公園,17740


In [10]:
towada = tourism_train.loc[tourism_train["park"] == "十和田八幡平国立公園",]
print(towada.shape)
towada.to_csv("../data/towada.csv",index=False)
towada.head()


(731, 3)


,datetime,park,visitors
1,2015-01-01,十和田八幡平国立公園,34757
9,2015-01-02,十和田八幡平国立公園,33795
17,2015-01-03,十和田八幡平国立公園,17114
25,2015-01-04,十和田八幡平国立公園,9306
33,2015-01-05,十和田八幡平国立公園,4732


In [11]:
nikko = tourism_train.loc[tourism_train["park"] == "日光国立公園",]
print(nikko.shape)
nikko.to_csv("../data/nikko.csv",index=False)
nikko.head()


(731, 3)


,datetime,park,visitors
2,2015-01-01,日光国立公園,29714
10,2015-01-02,日光国立公園,27011
18,2015-01-03,日光国立公園,22247
26,2015-01-04,日光国立公園,12236
34,2015-01-05,日光国立公園,6571


In [12]:
akan = tourism_train.loc[tourism_train["park"] == "阿寒摩周国立公園",]
print(akan.shape)
akan.to_csv("../data/akan.csv",index=False)
akan.head()


(731, 3)


,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
8,2015-01-02,阿寒摩周国立公園,11153
16,2015-01-03,阿寒摩周国立公園,12343
24,2015-01-04,阿寒摩周国立公園,6732
32,2015-01-05,阿寒摩周国立公園,4877


In [13]:
ooyama = tourism_train.loc[tourism_train["park"] == "大山隠岐国立公園",]
print(ooyama.shape)
ooyama.to_csv("../data/ooyama.csv",index=False)
ooyama.head()


(731, 3)


,datetime,park,visitors
4,2015-01-01,大山隠岐国立公園,3637
12,2015-01-02,大山隠岐国立公園,3244
20,2015-01-03,大山隠岐国立公園,1927
28,2015-01-04,大山隠岐国立公園,880
36,2015-01-05,大山隠岐国立公園,135


In [14]:
aso = tourism_train.loc[tourism_train["park"] == "阿蘇くじゅう国立公園",]
print(aso.shape)
aso.to_csv("../data/aso.csv",index=False)
aso.head()


(731, 3)


,datetime,park,visitors
5,2015-01-01,阿蘇くじゅう国立公園,1369
13,2015-01-02,阿蘇くじゅう国立公園,1643
21,2015-01-03,阿蘇くじゅう国立公園,967
29,2015-01-04,阿蘇くじゅう国立公園,324
37,2015-01-05,阿蘇くじゅう国立公園,100


In [15]:
kirisima = tourism_train.loc[tourism_train["park"] == "霧島錦江湾国立公園",]
print(kirisima.shape)
kirisima.to_csv("../data/kirisima.csv",index=False)
kirisima.head()


(731, 3)


,datetime,park,visitors
6,2015-01-01,霧島錦江湾国立公園,35352
14,2015-01-02,霧島錦江湾国立公園,29856
22,2015-01-03,霧島錦江湾国立公園,27027
30,2015-01-04,霧島錦江湾国立公園,12993
38,2015-01-05,霧島錦江湾国立公園,8112


In [16]:
kerama = tourism_train.loc[tourism_train["park"] == "慶良間諸島国立公園",]
print(kerama.shape)
kerama.to_csv("../data/kerama.csv",index=False)
kerama.head()

(731, 3)


,datetime,park,visitors
7,2015-01-01,慶良間諸島国立公園,151
15,2015-01-02,慶良間諸島国立公園,657
23,2015-01-03,慶良間諸島国立公園,126
31,2015-01-04,慶良間諸島国立公園,75
39,2015-01-05,慶良間諸島国立公園,0


In [19]:
lag = 8
target = "aso"

data = pd.DataFrame()
data[target] = pd.read_csv("../data/"+target+".csv")["visitors"]
data.index = pd.to_datetime(pd.read_csv("../data/"+target+".csv")["datetime"])
data.head(20)

,aso
datetime,
2015-01-01,1369
2015-01-02,1643
2015-01-03,967
2015-01-04,324
2015-01-05,100
2015-01-06,0
2015-01-07,25
2015-01-08,0
2015-01-09,50


In [22]:
feature = pd.DataFrame()
for l in list(range(1,lag+1)):
    feature.loc[:,target + "lag"+str(l)] = data[target].shift(-(lag-l))
feature.loc[:,"y"] = data[target].shift(-lag)
feature.index = feature.index + offsets.Day(lag)
feature

,asolag1,asolag2,asolag3,asolag4,asolag5,asolag6,asolag7,asolag8,y
datetime,,,,,,,,,
2015-01-09,0.0,25.0,0.0,100.0,324.0,967.0,1643.0,1369,50.0
2015-01-10,50.0,0.0,25.0,0.0,100.0,324.0,967.0,1643,448.0
2015-01-11,448.0,50.0,0.0,25.0,0.0,100.0,324.0,967,174.0
2015-01-12,174.0,448.0,50.0,0.0,25.0,0.0,100.0,324,0.0
2015-01-13,0.0,174.0,448.0,50.0,0.0,25.0,0.0,100,50.0
2015-01-14,50.0,0.0,174.0,448.0,50.0,0.0,25.0,0,50.0
2015-01-15,50.0,50.0,0.0,174.0,448.0,50.0,0.0,25,100.0
2015-01-16,100.0,50.0,50.0,0.0,174.0,448.0,50.0,0,446.0
2015-01-17,446.0,100.0,50.0,50.0,0.0,174.0,448.0,50,347.0


In [23]:
feature.to_csv("../data/"+target+"l"+str(lag)+".csv")

In [24]:
lag = 8
targets = ["aso","akan","isesima","ooyama","kirisima","kerama","towada","nikko"]

for target in targets:
    data = pd.DataFrame()
    data[target] = pd.read_csv("../data/"+target+".csv")["visitors"]
    data.index = pd.to_datetime(pd.read_csv("../data/"+target+".csv")["datetime"])
    
    feature = pd.DataFrame()
    for l in list(range(1,lag+1)):
        feature.loc[:,target + "lag"+str(l)] = data[target].shift(-(lag-l))
    feature.loc[:,"y"] = data[target].shift(-lag)
    feature.index = feature.index + offsets.Day(lag)
    display(feature)
    feature.to_csv("../data/"+target+"l"+str(lag)+".csv")

,asolag1,asolag2,asolag3,asolag4,asolag5,asolag6,asolag7,asolag8,y
datetime,,,,,,,,,
2015-01-09,0.0,25.0,0.0,100.0,324.0,967.0,1643.0,1369,50.0
2015-01-10,50.0,0.0,25.0,0.0,100.0,324.0,967.0,1643,448.0
2015-01-11,448.0,50.0,0.0,25.0,0.0,100.0,324.0,967,174.0
2015-01-12,174.0,448.0,50.0,0.0,25.0,0.0,100.0,324,0.0
2015-01-13,0.0,174.0,448.0,50.0,0.0,25.0,0.0,100,50.0
2015-01-14,50.0,0.0,174.0,448.0,50.0,0.0,25.0,0,50.0
2015-01-15,50.0,50.0,0.0,174.0,448.0,50.0,0.0,25,100.0
2015-01-16,100.0,50.0,50.0,0.0,174.0,448.0,50.0,0,446.0
2015-01-17,446.0,100.0,50.0,50.0,0.0,174.0,448.0,50,347.0


,akanlag1,akanlag2,akanlag3,akanlag4,akanlag5,akanlag6,akanlag7,akanlag8,y
datetime,,,,,,,,,
2015-01-09,2715.0,4569.0,4162.0,4877.0,6732.0,12343.0,11153.0,11028,2258.0
2015-01-10,2258.0,2715.0,4569.0,4162.0,4877.0,6732.0,12343.0,11153,4273.0
2015-01-11,4273.0,2258.0,2715.0,4569.0,4162.0,4877.0,6732.0,12343,4273.0
2015-01-12,4273.0,4273.0,2258.0,2715.0,4569.0,4162.0,4877.0,6732,2686.0
2015-01-13,2686.0,4273.0,4273.0,2258.0,2715.0,4569.0,4162.0,4877,2439.0
2015-01-14,2439.0,2686.0,4273.0,4273.0,2258.0,2715.0,4569.0,4162,1545.0
2015-01-15,1545.0,2439.0,2686.0,4273.0,4273.0,2258.0,2715.0,4569,1455.0
2015-01-16,1455.0,1545.0,2439.0,2686.0,4273.0,4273.0,2258.0,2715,1595.0
2015-01-17,1595.0,1455.0,1545.0,2439.0,2686.0,4273.0,4273.0,2258,2545.0


,isesimalag1,isesimalag2,isesimalag3,isesimalag4,isesimalag5,isesimalag6,isesimalag7,isesimalag8,y
datetime,,,,,,,,,
2015-01-09,3489.0,4612.0,9118.0,17740.0,34960.0,45368.0,47035.0,42652,5327.0
2015-01-10,5327.0,3489.0,4612.0,9118.0,17740.0,34960.0,45368.0,47035,17676.0
2015-01-11,17676.0,5327.0,3489.0,4612.0,9118.0,17740.0,34960.0,45368,21679.0
2015-01-12,21679.0,17676.0,5327.0,3489.0,4612.0,9118.0,17740.0,34960,10215.0
2015-01-13,10215.0,21679.0,17676.0,5327.0,3489.0,4612.0,9118.0,17740,4297.0
2015-01-14,4297.0,10215.0,21679.0,17676.0,5327.0,3489.0,4612.0,9118,4489.0
2015-01-15,4489.0,4297.0,10215.0,21679.0,17676.0,5327.0,3489.0,4612,5178.0
2015-01-16,5178.0,4489.0,4297.0,10215.0,21679.0,17676.0,5327.0,3489,5818.0
2015-01-17,5818.0,5178.0,4489.0,4297.0,10215.0,21679.0,17676.0,5327,14964.0


,ooyamalag1,ooyamalag2,ooyamalag3,ooyamalag4,ooyamalag5,ooyamalag6,ooyamalag7,ooyamalag8,y
datetime,,,,,,,,,
2015-01-09,270.0,542.0,141.0,135.0,880.0,1927.0,3244.0,3637,88.0
2015-01-10,88.0,270.0,542.0,141.0,135.0,880.0,1927.0,3244,384.0
2015-01-11,384.0,88.0,270.0,542.0,141.0,135.0,880.0,1927,613.0
2015-01-12,613.0,384.0,88.0,270.0,542.0,141.0,135.0,880,96.0
2015-01-13,96.0,613.0,384.0,88.0,270.0,542.0,141.0,135,352.0
2015-01-14,352.0,96.0,613.0,384.0,88.0,270.0,542.0,141,310.0
2015-01-15,310.0,352.0,96.0,613.0,384.0,88.0,270.0,542,85.0
2015-01-16,85.0,310.0,352.0,96.0,613.0,384.0,88.0,270,239.0
2015-01-17,239.0,85.0,310.0,352.0,96.0,613.0,384.0,88,260.0


,kirisimalag1,kirisimalag2,kirisimalag3,kirisimalag4,kirisimalag5,kirisimalag6,kirisimalag7,kirisimalag8,y
datetime,,,,,,,,,
2015-01-09,8404.0,16559.0,7856.0,8112.0,12993.0,27027.0,29856.0,35352,10997.0
2015-01-10,10997.0,8404.0,16559.0,7856.0,8112.0,12993.0,27027.0,29856,28202.0
2015-01-11,28202.0,10997.0,8404.0,16559.0,7856.0,8112.0,12993.0,27027,26121.0
2015-01-12,26121.0,28202.0,10997.0,8404.0,16559.0,7856.0,8112.0,12993,15286.0
2015-01-13,15286.0,26121.0,28202.0,10997.0,8404.0,16559.0,7856.0,8112,8633.0
2015-01-14,8633.0,15286.0,26121.0,28202.0,10997.0,8404.0,16559.0,7856,10129.0
2015-01-15,10129.0,8633.0,15286.0,26121.0,28202.0,10997.0,8404.0,16559,13873.0
2015-01-16,13873.0,10129.0,8633.0,15286.0,26121.0,28202.0,10997.0,8404,9914.0
2015-01-17,9914.0,13873.0,10129.0,8633.0,15286.0,26121.0,28202.0,10997,14541.0


,keramalag1,keramalag2,keramalag3,keramalag4,keramalag5,keramalag6,keramalag7,keramalag8,y
datetime,,,,,,,,,
2015-01-09,25.0,0.0,0.0,0.0,75.0,126.0,657.0,151,0.0
2015-01-10,0.0,25.0,0.0,0.0,0.0,75.0,126.0,657,510.0
2015-01-11,510.0,0.0,25.0,0.0,0.0,0.0,75.0,126,510.0
2015-01-12,510.0,510.0,0.0,25.0,0.0,0.0,0.0,75,50.0
2015-01-13,50.0,510.0,510.0,0.0,25.0,0.0,0.0,0,0.0
2015-01-14,0.0,50.0,510.0,510.0,0.0,25.0,0.0,0,0.0
2015-01-15,0.0,0.0,50.0,510.0,510.0,0.0,25.0,0,0.0
2015-01-16,0.0,0.0,0.0,50.0,510.0,510.0,0.0,25,0.0
2015-01-17,0.0,0.0,0.0,0.0,50.0,510.0,510.0,0,0.0


,towadalag1,towadalag2,towadalag3,towadalag4,towadalag5,towadalag6,towadalag7,towadalag8,y
datetime,,,,,,,,,
2015-01-09,3019.0,2374.0,2771.0,4732.0,9306.0,17114.0,33795.0,34757,3865.0
2015-01-10,3865.0,3019.0,2374.0,2771.0,4732.0,9306.0,17114.0,33795,16346.0
2015-01-11,16346.0,3865.0,3019.0,2374.0,2771.0,4732.0,9306.0,17114,16169.0
2015-01-12,16169.0,16346.0,3865.0,3019.0,2374.0,2771.0,4732.0,9306,7301.0
2015-01-13,7301.0,16169.0,16346.0,3865.0,3019.0,2374.0,2771.0,4732,2534.0
2015-01-14,2534.0,7301.0,16169.0,16346.0,3865.0,3019.0,2374.0,2771,2425.0
2015-01-15,2425.0,2534.0,7301.0,16169.0,16346.0,3865.0,3019.0,2374,2365.0
2015-01-16,2365.0,2425.0,2534.0,7301.0,16169.0,16346.0,3865.0,3019,3237.0
2015-01-17,3237.0,2365.0,2425.0,2534.0,7301.0,16169.0,16346.0,3865,5982.0


,nikkolag1,nikkolag2,nikkolag3,nikkolag4,nikkolag5,nikkolag6,nikkolag7,nikkolag8,y
datetime,,,,,,,,,
2015-01-09,1548.0,1031.0,3593.0,6571.0,12236.0,22247.0,27011.0,29714,1216.0
2015-01-10,1216.0,1548.0,1031.0,3593.0,6571.0,12236.0,22247.0,27011,11152.0
2015-01-11,11152.0,1216.0,1548.0,1031.0,3593.0,6571.0,12236.0,22247,11820.0
2015-01-12,11820.0,11152.0,1216.0,1548.0,1031.0,3593.0,6571.0,12236,3454.0
2015-01-13,3454.0,11820.0,11152.0,1216.0,1548.0,1031.0,3593.0,6571,665.0
2015-01-14,665.0,3454.0,11820.0,11152.0,1216.0,1548.0,1031.0,3593,680.0
2015-01-15,680.0,665.0,3454.0,11820.0,11152.0,1216.0,1548.0,1031,2722.0
2015-01-16,2722.0,680.0,665.0,3454.0,11820.0,11152.0,1216.0,1548,1933.0
2015-01-17,1933.0,2722.0,680.0,665.0,3454.0,11820.0,11152.0,1216,4553.0
